In [5]:
import pickle

countVector = pickle.load( open( "files/countVector.pickle", "rb" ) )

In [6]:
countVector

<8430264x430 sparse matrix of type '<class 'numpy.int64'>'
	with 107249548 stored elements in Compressed Sparse Row format>

In [8]:
import pandas as pd
import numpy as np

df = pickle.load( open( "files/df.pickle", "rb" ) )
df['Tweet'] = df['Tweet'].astype(str)
df['Len'] = df['Tweet'].str.split().apply(len)
df = df[df['Len']>=4]
print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8430264
Russian 2283309


In [9]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(countVector, df['Category'])

In [10]:
model = LatentDirichletAllocation(75)
model.fit(X_train)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=75, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
XX_train = model.transform(X_train)
XX_test = model.transform(X_test)

In [ ]:
pickle.dump( model, open( "models/lda_model.pickle", "wb" ) )
pickle.dump( X_test, open( "files/X_test.pickle", "wb" ) )
pickle.dump( X_train, open( "files/X_train.pickle", "wb" ) )
pickle.dump( X_test, open( "files/XX_test.pickle", "wb" ) )
pickle.dump( X_train, open( "files/XX_train.pickle", "wb" ) )

In [ ]:
pickle.dump( y_train, open( "files/y_train.pickle", "wb" ) )
pickle.dump( y_test, open( "files/y_test.pickle", "wb" ) )

In [ ]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = nltk.word_tokenize(text_rc)   # tokenization
    tokens2 = [a[0] if a[1][0] is not 'N' else a[1] for a in nltk.pos_tag(tokens)]
    tokens = tokens + ['','','','',''] + tokens2
    text = [ps.stem(word) for word in tokens]
    return text

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(model, countVector, pickle.load( open( "files/countVectorizer.pickle", "rb" ) ), sort_topics=False)